In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

# Load data from Excel file
data_path = '../data_latih.xlsx'
data_latih = pd.read_excel(data_path)

# Encoding data kategorikal
le = LabelEncoder()

# Encode 'L/P'
data_latih['L/P'] = le.fit_transform(data_latih['L/P'])

# Encode 'Penghasilan'
penghasilan_mapping = {
    'Rp. 500,000 - Rp. 999,999': 0,
    'Rp. 1,000,000 - Rp. 1,999,999': 1,
    'Rp. 2,000,000 - Rp. 4,999,999': 2,
    'Tidak Berpenghasilan': 3,
    'Kurang dari Rp. 500,000': 4
}
data_latih['Penghasilan'] = data_latih['Penghasilan'].map(penghasilan_mapping)

# Encode 'Status Ekonomi'
status_ekonomi_mapping = {
    'SANGAT MISKIN': 0,
    'MISKIN': 1,
    'CUKUP': 2
}
data_latih['Status Ekonomi'] = data_latih['Status Ekonomi'].map(status_ekonomi_mapping)

# Encode 'Layak PIP'
layak_pip_mapping = {'Ya': 1, 'Tidak': 0}
data_latih['Layak PIP'] = data_latih['Layak PIP'].map(layak_pip_mapping)
# Drop columns that are not used in the model
data_latih = data_latih.drop(columns=['Nama', 'Alasan Layak PIP', 'Status Bantuan', 'Status Kesesuaian'])

# Pisahkan fitur dan target
X = data_latih.drop(columns=['Layak PIP'])
y = data_latih['Layak PIP']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan shapes dari training dan testing set
print("Training set shape:")
print(X_train.shape)
print(y_train.shape)
print("Testing set shape:")
print(X_test.shape)
print(y_test.shape)



In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisasi kolom numerik
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Display the first few rows of the scaled data
print("First few rows of X_train_scaled:")
display(X_train_scaled.head())
print("First few rows of X_test_scaled:")
display(X_test_scaled.head())



In [ ]:
# Inisialisasi parameter

# Bobot awal diinisialisasi dengan nilai nol untuk semua fitur
w = np.zeros(X_train_scaled.shape[1])

# Bias awal diinisialisasi dengan nilai nol
b = 0

# Learning rate (alpha) menentukan seberapa besar langkah yang diambil pada setiap iterasi saat memperbarui bobot dan bias
learning_rate = 0.01

# Jumlah iterasi menentukan berapa kali proses pembaruan bobot dan bias dilakukan
n_iterations = 1000

# Menampilkan parameter awal
print("Initial weights (w):", w)
print("Initial bias (b):", b)
print("Learning rate:", learning_rate)
print("Number of iterations:", n_iterations)



In [ ]:
# Fungsi untuk menghitung hinge loss
def hinge_loss(X, y, w, b):
    # Hinge loss dihitung dengan maks(0, 1 - y * (X * w + b))
    return np.maximum(0, 1 - y * (np.dot(X, w) + b))

# Menghitung hinge loss untuk parameter awal
X_train_array = X_train_scaled.to_numpy()
y_train_array = y_train.to_numpy()

initial_loss = hinge_loss(X_train_array, y_train_array, w, b)
print("Initial hinge loss:", initial_loss)


In [ ]:
def gradient_descent(X, y, w, b, learning_rate, n_iterations):
    # Melakukan iterasi sebanyak n_iterations
    for iteration in range(n_iterations):
        # Iterasi melalui setiap sampel data
        for idx, x_i in enumerate(X):
            # Menghitung kondisi apakah sampel terklasifikasi dengan benar (margin >= 1)
            if y[idx] * (np.dot(x_i, w) + b) >= 1:
                # Jika terklasifikasi dengan benar, hanya terapkan regularisasi pada bobot
                w -= learning_rate * (2 * 1/n_iterations * w)
            else:
                # Jika terklasifikasi salah, perbarui bobot dan bias untuk mengurangi kesalahan
                w -= learning_rate * (2 * 1/n_iterations * w - np.dot(x_i, y[idx] * x_i))
                b -= learning_rate * y[idx]
        
        # Cetak bobot dan bias pada setiap iterasi
        print(f"Iteration {iteration + 1}/{n_iterations}:")
        print(f"Weights: {w}")
        print(f"Bias: {b}\n")
    
    # Mengembalikan bobot dan bias yang telah diperbarui
    return w, b

# Inisialisasi parameter
w = np.zeros(X_train_scaled.shape[1])  # Bobot awal diinisialisasi dengan nilai nol untuk semua fitur
b = 0  # Bias awal diinisialisasi dengan nilai nol
learning_rate = 0.01  # Learning rate menentukan seberapa besar langkah yang diambil pada setiap iterasi
n_iterations = 100  # Jumlah iterasi menentukan berapa kali proses pembaruan bobot dan bias dilakukan (gunakan 10 untuk mencetak lebih sedikit iterasi)

# Melakukan gradient descent untuk memperbarui bobot dan bias
w, b = gradient_descent(X_train_scaled.to_numpy(), y_train.to_numpy(), w, b, learning_rate, n_iterations)

# Menampilkan parameter setelah pelatihan
print("Trained weights (w):", w)
print("Trained bias (b):", b)




In [ ]:
# Fungsi untuk melakukan prediksi menggunakan model SVM yang telah dilatih
def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)
# Melakukan prediksi pada data uji
y_pred = predict(X_test_scaled.to_numpy(), w, b)

# Menghitung akurasi model
accuracy = np.mean(y_pred == y_test.to_numpy())

print(f'Accuracy: {accuracy}')



In [ ]:
# Melakukan prediksi pada data latih
y_train_pred = predict(X_train_scaled.to_numpy(), w, b)

# Menghitung akurasi model pada data latih
train_accuracy = np.mean(y_train_pred == y_train.to_numpy())

print(f'Train Accuracy: {train_accuracy}')


In [ ]:
import joblib

# Simpan bobot (weights) dan bias model ke file
model_path = 'svm_model.pkl'
joblib.dump((w, b), model_path)

print(f"Model saved to {model_path}")


In [ ]:
import joblib

# Muat bobot (weights) dan bias model dari file
w_loaded, b_loaded = joblib.load(model_path)

print("Model loaded:")
print(f"Weights: {w_loaded}")
print(f"Bias: {b_loaded}")


In [ ]:
# Muat data uji dari file Excel
data_uji_path = '../data_uji.xlsx'
data_uji = pd.read_excel(data_uji_path)

# Tampilkan beberapa baris pertama dari data uji
display(data_uji.head())


In [ ]:
# Fungsi untuk melakukan prediksi menggunakan model SVM yang telah dilatih
def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

# Normalisasi data uji (asumsi kolom yang digunakan sama dengan data latih)
X_uji = data_uji.drop(columns=['Layak PIP'])
scaler = StandardScaler()
X_uji_scaled = scaler.fit_transform(X_uji)

# Lakukan prediksi pada data uji
y_pred_uji = predict(X_uji_scaled, w_loaded, b_loaded)

In [ ]:
# Konversi hasil prediksi ke status kelayakan
status_kelayakan = pd.Series(y_pred_uji).map({1: 'Sesuai', -1: 'Tidak Layak'})

# Jika prediksi berbeda dengan keterangan "Layak PIP", berikan status "Tidak Tepat Sasaran"
data_uji['Prediksi'] = status_kelayakan
data_uji['Status Kelayakan'] = data_uji.apply(
    lambda row: 'Sesuai' if row['Prediksi'] == 'Sesuai' and row['Layak PIP'] == 1 else
                'Tidak Layak' if row['Prediksi'] == 'Tidak Layak' and row['Layak PIP'] == 0 else
                'Tidak Tepat Sasaran', axis=1
)


In [ ]:
# Tampilkan hasil prediksi dan keterangan sebenarnya
comparison = data_uji[['Layak PIP', 'Prediksi', 'Status Kelayakan']]

# Tampilkan perbandingan
display(comparison)


In [ ]:
# Hitung jumlah yang sesuai atau tidak sesuai
jumlah_sesuai = data_uji['Status Kelayakan'].value_counts().get('Sesuai', 0)
jumlah_tidak_layak = data_uji['Status Kelayakan'].value_counts().get('Tidak Layak', 0)
jumlah_tidak_tepat_sasaran = data_uji['Status Kelayakan'].value_counts().get('Tidak Tepat Sasaran', 0)

print(f"Jumlah Sesuai: {jumlah_sesuai}")
print(f"Jumlah Tidak Layak: {jumlah_tidak_layak}")
print(f"Jumlah Tidak Tepat Sasaran: {jumlah_tidak_tepat_sasaran}")


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

# Inisialisasi model SVM Linier
svm_model = SVC(kernel='linear')

# Melatih model
svm_model.fit(X_train_scaled, y_train)

# Simpan model SVM Linier ke file
svm_model_path = 'svm_model_no_adaBoost.pkl'
joblib.dump(svm_model, svm_model_path)
print(f"Model SVM Linier disimpan ke {svm_model_path}")

# Melakukan prediksi pada data uji
y_pred_svm = svm_model.predict(X_test_scaled)

# Menghitung akurasi model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Accuracy SVM Linier: {accuracy_svm}')





In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Inisialisasi model AdaBoost dengan base estimator SVM
adaboost_model = AdaBoostClassifier(base_estimator=SVC(kernel='linear', probability=True), n_estimators=50, random_state=42)

# Melatih model
adaboost_model.fit(X_train_scaled, y_train)

# Simpan model SVM dengan AdaBoost ke file
adaboost_model_path = 'adaboost_svm_model.pkl'
joblib.dump(adaboost_model, adaboost_model_path)
print(f"Model SVM dengan AdaBoost disimpan ke {adaboost_model_path}")

# Melakukan prediksi pada data uji
y_pred_adaboost = adaboost_model.predict(X_test_scaled)

# Menghitung akurasi model
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print(f'Accuracy SVM dengan AdaBoost: {accuracy_adaboost}')


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix dan laporan klasifikasi untuk SVM Linier
print("Confusion Matrix - SVM Linier")
print(confusion_matrix(y_test, y_pred_svm))
print("\nClassification Report - SVM Linier")
print(classification_report(y_test, y_pred_svm))

# Confusion matrix dan laporan klasifikasi untuk SVM dengan AdaBoost
print("Confusion Matrix - SVM dengan AdaBoost")
print(confusion_matrix(y_test, y_pred_adaboost))
print("\nClassification Report - SVM dengan AdaBoost")
print(classification_report(y_test, y_pred_adaboost))


In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Langkah 1: Muat dan siapkan data latih
data_latih_path = '../data_latih.xlsx'
data_latih = pd.read_excel(data_latih_path)

# Encoding data kategorikal
le = LabelEncoder()
data_latih['L/P'] = le.fit_transform(data_latih['L/P'])
penghasilan_mapping = {
    'Rp. 500,000 - Rp. 999,999': 0,
    'Rp. 1,000,000 - Rp. 1,999,999': 1,
    'Rp. 2,000,000 - Rp. 4,999,999': 2,
    'Tidak Berpenghasilan': 3,
    'Kurang dari Rp. 500,000': 4
}
data_latih['Penghasilan'] = data_latih['Penghasilan'].map(penghasilan_mapping)
status_ekonomi_mapping = {
    'SANGAT MISKIN': 0,
    'MISKIN': 1,
    'CUKUP': 2
}
data_latih['Status Ekonomi'] = data_latih['Status Ekonomi'].map(status_ekonomi_mapping)
layak_pip_mapping = {'Ya': 1, 'Tidak': 0}
data_latih['Layak PIP'] = data_latih['Layak PIP'].map(layak_pip_mapping)

# Drop kolom yang tidak digunakan
data_latih = data_latih.drop(columns=['Nama', 'Alasan Layak PIP', 'Status Bantuan', 'Status Kesesuaian'])

# Pisahkan fitur dan target
X = data_latih.drop(columns=['Layak PIP'])
y = data_latih['Layak PIP']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Langkah 2: Melatih model Decision Tree dan model Decision Tree dengan AdaBoost
tree_model = DecisionTreeClassifier()
adaboost_model = AdaBoostClassifier(base_estimator=tree_model, n_estimators=50, algorithm='SAMME')

# Melatih model
tree_model.fit(X_train_scaled, y_train)
adaboost_model.fit(X_train_scaled, y_train)

# Simpan model ke file
joblib.dump(tree_model, 'tree_model.pkl')
joblib.dump(adaboost_model, 'adaboost_tree_model.pkl')

# Langkah 3: Muat model dari file
tree_model = joblib.load('tree_model.pkl')
adaboost_model = joblib.load('adaboost_tree_model.pkl')

# Muat data uji dari file Excel
data_uji_path = '../data_uji.xlsx'
data_uji = pd.read_excel(data_uji_path)

# Normalisasi data uji (asumsi kolom yang digunakan sama dengan data latih)
X_uji = data_uji.drop(columns=['Layak PIP'])
X_uji_scaled = scaler.transform(X_uji)

# Target data uji
y_uji = data_uji['Layak PIP']

# Lakukan prediksi pada data uji menggunakan Decision Tree
y_pred_tree = tree_model.predict(X_uji_scaled)

# Lakukan prediksi pada data uji menggunakan Decision Tree dengan AdaBoost
y_pred_adaboost = adaboost_model.predict(X_uji_scaled)

# Menghitung akurasi model
accuracy_tree = accuracy_score(y_uji, y_pred_tree)
accuracy_adaboost = accuracy_score(y_uji, y_pred_adaboost)

# Membuat DataFrame untuk menampilkan hasil akurasi
accuracy_df = pd.DataFrame({
    'Model': ['Decision Tree', 'Decision Tree dengan AdaBoost'],
    'Accuracy': [accuracy_tree, accuracy_adaboost]
})

# Membuat DataFrame untuk confusion matrix
confusion_matrix_tree = confusion_matrix(y_uji, y_pred_tree)
confusion_matrix_adaboost = confusion_matrix(y_uji, y_pred_adaboost)

confusion_df_tree = pd.DataFrame(confusion_matrix_tree, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])
confusion_df_adaboost = pd.DataFrame(confusion_matrix_adaboost, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])

# Membuat DataFrame untuk laporan klasifikasi
classification_report_tree = classification_report(y_uji, y_pred_tree, output_dict=True)
classification_report_adaboost = classification_report(y_uji, y_pred_adaboost, output_dict=True)

classification_df_tree = pd.DataFrame(classification_report_tree).transpose()
classification_df_adaboost = pd.DataFrame(classification_report_adaboost).transpose()

# Menampilkan hasil dalam bentuk tabel
print("Accuracy Comparison:")
display(accuracy_df)

print("\nConfusion Matrix - Decision Tree:")
display(confusion_df_tree)

print("\nConfusion Matrix - Decision Tree dengan AdaBoost:")
display(confusion_df_adaboost)

print("\nClassification Report - Decision Tree:")
display(classification_df_tree)

print("\nClassification Report - Decision Tree dengan AdaBoost:")
display(classification_df_adaboost)
